# SEIRX model runs for test technology and screening frequency investigation

In [9]:
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
import data_creation_functions as dcf

In [10]:
# paths for I/O
contact_network_src = '../data/contact_networks'

## Simulation function

In [11]:
def run(param_list):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''

    # extract the simulation parameters from the parameter list
    index_case, e_vaccination_ratio, r_vaccination_ratio, N_runs = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, measures, simulation_params,
                contact_network_src, ensmbl_dst, index_case=index_case, 
                e_vaccination_ratio=e_vaccination_ratio,
                r_vaccination_ratio=r_vaccination_ratio)

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results, index_case,
                                e_vaccination_ratio=e_vaccination_ratio,
                                r_vaccination_ratio=r_vaccination_ratio)

    return row

## Screening parameters

In [12]:
N_runs = 1000

e_vaccination_ratios = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
r_vaccination_ratios = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# specifies, whether the index case will be introduced via an
# employee or a resident
index_cases = ['employee', 'resident']

screening_params = [(i, j, k, N_runs)\
              for i in index_cases \
              for j in e_vaccination_ratios \
              for k in r_vaccination_ratios]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 242 parameter combinations


## Wild type

In [8]:
ensmbl_dst = '../data/simulation_results/ensembles/vaccination/wild_type'

### Simulation parameters and measures in place

In [ ]:
with open('params/vaccination_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/vaccination_measures.json') as json_file:
    measures = json.load(json_file)


### Simulation runs

In [ ]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 2 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case',
              'resident_vaccination_ratio', 'employee_vaccination_ratio']

other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

dst = '../data/simulation_results'
results.to_csv(join(dst,'simulations_vaccination_rate_{}.csv'\
                   .format(N_runs)), index=False)

results.head(3)

## B.1.1.7

### Simulation parameters and measures in place

In [13]:
with open('params/vaccination_UK_variant_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/vaccination_measures.json') as json_file:
    measures = json.load(json_file)


### Simulation runs

In [ ]:
%%time
ensmbl_dst = '../data/simulation_results/ensembles/vaccination/UK_variant'

number_of_cores = psutil.cpu_count(logical=True) - 2 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case',
              'resident_vaccination_ratio', 'employee_vaccination_ratio']

other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

dst = '../data/simulation_results'
results.to_csv(join(dst,'simulations_vaccination_rate_UK_variant_{}.csv'\
                   .format(N_runs)), index=False)

results.head(3)

 23%|██▎       | 55/242 [20:04<47:08, 15.13s/it]  